In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import vmodel.random as vrandom
import vmodel.liveplot as vplot
import vmodel.geometry as vgeom
import vmodel.visibility as vvis
import vmodel.util.color as vcolor

from scipy.spatial import Voronoi, Delaunay, ConvexHull

from vmodel.plot import voronoi_plot_2d

%load_ext autoreload
%autoreload 2

In [ ]:
num_agents = 200
radius_agent = 0.25
radius_arena = 10
radius_perc = 3
min_dist = 1
maxlim = radius_arena + radius_agent * 2
k = np.sqrt(2)
lim = (-radius_perc * k, radius_perc * k)

In [ ]:
# positions = vrandom.poisson_disk_spherical(radius_arena, min_dist, num_agents, candidate='first')
# positions.shape

In [ ]:
# Save random positions to CSV for reproducibility
# np.savetxt('agent_positions.csv', positions)

In [ ]:
positions = np.loadtxt('agent_positions.csv')

In [ ]:
# Find pos self!
index = [i for i in range(len(positions)) if positions[i][0] == 0][0]
index

In [ ]:
pos_self = positions[index]
pos_others = np.delete(positions, index, axis=0)

In [ ]:
visibility = vvis.visibility_set(pos_others, radius_agent)
visibility.shape

In [ ]:
max_agents = 6
distances = np.linalg.norm(pos_others, axis=1)
indices = distances.argsort()[:max_agents]

In [ ]:
color_gray = vcolor.grey
color_blue = vcolor.blue
color_lgray = vcolor.lightgrey
figsize = (5, 5)

# Metric

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

# Add background rectangle
xy, size = (-maxlim, -maxlim), 2 * maxlim
background = plt.Rectangle(xy, size, size, color=color_lgray)
ax.add_patch(background)

perc_radius = plt.Circle((0, 0), radius=radius_perc, color='white')
ax.add_patch(perc_radius)

perc_circle = plt.Circle((0, 0), radius=radius_perc, fill=False, ls=':', lw=0.5, ec='grey', zorder=100)
ax.add_patch(perc_circle)

for i, pos in enumerate(pos_others):

    too_far = distances[i] > radius_perc
    color = color_gray if too_far else color_blue

    vplot.plot_circle(ax, pos, radius=radius_agent, color=color, zorder=2, fill=True)
vplot.plot_circle(ax, (0, 0), radius=radius_agent, color='tab:red', zorder=99)
ax.set(aspect='equal')
ax.set(xlim=lim, ylim=lim)
ax.set(xticks=[], yticks=[])
fig.savefig(f'1_neighbor_selection_metric.pdf', bbox_inches='tight')

# Visual

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
scale = 100

# Add background rectangle
xy, size = (-maxlim, -maxlim), 2 * maxlim

# Add background rectangle
xy, size = (-maxlim, -maxlim), 2 * maxlim
background = plt.Rectangle(xy, size, size, color=color_lgray)
ax.add_patch(background)

perc_radius = plt.Circle((0, 0), radius=radius_perc, color='white')
ax.add_patch(perc_radius)

perc_circle = plt.Circle((0, 0), radius=radius_perc, fill=False, ls=':', lw=0.5, ec='grey', zorder=100)
ax.add_patch(perc_circle)

for i, pos in enumerate(pos_others):

    # Draw tangent points
    p1, p2 = vgeom.tangent_points_to_circle(pos, radius_agent)
    p1, p2 = np.array(p1), np.array(p2)

    ps1, ps2 = p1 * scale, p2 * scale
    d1, d2 = np.linalg.norm(p1), np.linalg.norm(p2)

    poly = np.array([p1, ps1, ps2, p2])
    polygon = plt.Polygon(poly, color=color_lgray, zorder=1)
    ax.add_patch(polygon)

    isvisible = visibility[i]
    isclose = distances[i] < radius_perc
    isneighbor = isvisible and isclose

    color = color_blue if isneighbor else color_gray

    vplot.plot_circle(ax, pos, radius=radius_agent, color=color, zorder=2, fill=True)
vplot.plot_circle(ax, (0, 0), radius=radius_agent, color='tab:red', zorder=99)
ax.set(aspect='equal')
ax.set(xlim=lim, ylim=lim)
ax.set(xticks=[], yticks=[])
fig.savefig(f'2_neighbor_selection_visual.pdf', bbox_inches='tight')

# Topological

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

# Add background rectangle
xy, size = (-maxlim, -maxlim), 2 * maxlim
background = plt.Rectangle(xy, size, size, color=color_lgray)
ax.add_patch(background)

# Sort by polar coordinates
pos_topo = [p for p in pos_others[indices]]
pos_topo.sort(key=lambda p: np.arctan2(p[1], p[0]))
polygon = plt.Polygon(pos_topo, color=vcolor.white)
border = plt.Polygon(pos_topo, color=vcolor.grey, ls=':', lw=0.5, fill=False)
ax.add_patch(polygon)
ax.add_patch(border)

for i, pos in enumerate(pos_others):

    include = (i in indices)

    color = color_blue if include else color_gray

    if include:
        x, y = pos
        
        ax.plot([0, x], [0, y], color=color_blue)

    vplot.plot_circle(ax, pos, radius=radius_agent, color=color, zorder=2, fill=True)
vplot.plot_circle(ax, (0, 0), radius=radius_agent, color=vcolor.focal, zorder=99)
ax.set(aspect='equal')
ax.set(xlim=lim, ylim=lim)
ax.set(xticks=[], yticks=[])
fig.savefig(f'3_neighbor_selection_topological.pdf', bbox_inches='tight')

# Voronoi

In [ ]:
fig, ax = plt.subplots(figsize=figsize)

pos_all = np.insert(pos_others, 0, np.zeros(2), axis=0)
vor = Voronoi(pos_all)
tri = Delaunay(pos_all)
neighbors = np.array(vgeom.voronoi_neighbors(pos_all)[0]) - 1
fig = voronoi_plot_2d(vor, ax=ax, show_vertices=False, point_size=0, line_colors=vcolor.grey, line_width=0.5, line_style=':')

# Color all non neighbor regions light grey
for index, r in enumerate(vor.point_region):
    region = vor.regions[r]
    if index - 1 in neighbors or index == 0:
        continue
    if not -1 in region:
        polygon = [vor.vertices[i] for i in region]
        ax.fill(*zip(*polygon), color=vcolor.lightgrey)

for i, pos in enumerate(pos_others):

    isneighbor = (i in neighbors)

    color = color_blue if isneighbor else color_gray
    vplot.plot_circle(ax, pos, radius=radius_agent, color=color, zorder=2, fill=True)
    
vplot.plot_circle(ax, (0, 0), radius=radius_agent, color=vcolor.focal, zorder=99)
ax.set(aspect='equal')
ax.set(xlim=lim, ylim=lim)
ax.set(xticks=[], yticks=[])
fig.savefig(f'4_neighbor_selection_voronoi.pdf', bbox_inches='tight')